In [11]:
# import os
tob3file = 'D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData\\Uploads\\20221202\\100hz\\100hz_Sensors_100Hz1.dat'
csvfile = tob3file.replace('.dat','.csv')
pklfile = os.path.join('D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData\Converted', os.path.basename(tob3file).replace('.dat','.pkl'))
for file in (tob3file, csvfile, pklfile):
    if os.path.isfile(file):
        print('%s exists' % file)
    else:
        print('%s does not exist' % file)

D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData\Uploads\20221202\100hz\100hz_Sensors_100Hz1.dat exists
D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData\Uploads\20221202\100hz\100hz_Sensors_100Hz1.csv exists
D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData\Converted\100hz_Sensors_100Hz1.pkl exists


In [30]:
import pandas as pd
dfcsv = pd.read_csv(csvfile, 
            #dtype={'TOA5':str, '100hz_Sensors':int, 'CR6':float, '18084':float, 
            #        'CR6.Std.12.01':float, 'CPU:VWIRE305_100hz.CR6':float, '20853':float, 'DynamicFreq':float}, 
            parse_dates=['TOA5'])
dfcsv.columns = dfcsv.iloc[0]
dfcsv=dfcsv.iloc[3:]
dfcsv['TIMESTAMP'] = pd.to_datetime(dfcsv.TIMESTAMP)
dfcsv['RECORD'] = dfcsv['RECORD'].astype(int)
#print(dfcsv.columns[2:])
for col in dfcsv.columns[2:]:
    dfcsv[col] = dfcsv[col].astype(float)
print(dfcsv)

C:\Users\thompsong\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0                     TIMESTAMP   RECORD   1226421   1226419   1226420  \
3       2022-11-04 12:00:00.010   690003  9165.072  9484.593  9705.107   
4       2022-11-04 12:00:00.020   690004  9164.785  9484.676  9705.748   
5       2022-11-04 12:00:00.030   690005  9165.193  9484.927  9705.301   
6       2022-11-04 12:00:00.040   690006  9165.072  9485.219  9705.704   
7       2022-11-04 12:00:00.050   690007  9164.985  9484.935  9705.896   
...                         ...      ...       ...       ...       ...   
1439998 2022-11-04 15:59:59.960  2129998  9170.681  9490.947  9707.750   
1439999 2022-11-04 15:59:59.970  2129999  9170.363  9490.942  9707.429   
1440000 2022-11-04 15:59:59.980  2130000  9170.893  9491.055  9707.063   
1440001 2022-11-04 15:59:59.990  2130001  9170.498  9490.857  9706.993   
1440002 2022-11-04 16:00:00.000  2130002  9170.743  9491.287  9707.424   

0         2149882   2151691   1226429  
3        8625.732  8354.872  9773.464  
4        8625.199  8355.347  97

In [36]:
try:
    dfpkl = pd.read_pickle(pklfile)
    print(dfpkl)
except:
    print('Cannot read pkl file. Probably written with a different version of pandas and cannot serialize')

Cannot read pkl file. Probably written with a different version of pandas and cannot serialize


In [31]:
import numpy as np
print('- Applying calibration equations')
WELLDATA_TOP = 'D:\Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\WellData'
transducersDF = pd.read_csv(os.path.join(WELLDATA_TOP, 'transducer_metadata.csv'))
def compute_psi(dig, d):
    psi = np.zeros((len(dig),1))
    #if np.isnan(d['dig0']):
    #    return psi
    for i in range(len(dig)):
        psi[i] = ((dig[i]-d['dig0']) * d['gf'] + (d['tt']-d['tt0'])*d['tf']+(d['bp0']-d['bp']))
        
    #print(level)
    return psi
df2 = dfcsv.copy()
for col in df2.columns:
    #print(col)
    if col[0:2]=='12' or col[0:2]=='21':
        this_transducer = transducersDF[(transducersDF['serial']) == col]
        #print(this_transducer)
        if len(this_transducer.index)==1:
            this_transducer = this_transducer.iloc[0].to_dict()
            #print(this_transducer)
            df2[col] = compute_psi(df2[col].to_numpy(), this_transducer)
print(df2)

- Applying calibration equations
0                     TIMESTAMP   RECORD   1226421   1226419    1226420  \
3       2022-11-04 12:00:00.010   690003  3.975500  3.365941  14.591738   
4       2022-11-04 12:00:00.020   690004  3.977430  3.365407  14.587599   
5       2022-11-04 12:00:00.030   690005  3.974687  3.363790  14.590486   
6       2022-11-04 12:00:00.040   690006  3.975500  3.361909  14.587883   
7       2022-11-04 12:00:00.050   690007  3.976085  3.363739  14.586643   
...                         ...      ...       ...       ...        ...   
1439998 2022-11-04 15:59:59.960  2129998  3.937785  3.325015  14.574670   
1439999 2022-11-04 15:59:59.970  2129999  3.939924  3.325048  14.576743   
1440000 2022-11-04 15:59:59.980  2130000  3.936360  3.324320  14.579107   
1440001 2022-11-04 15:59:59.990  2130001  3.939016  3.325595  14.579559   
1440002 2022-11-04 16:00:00.000  2130002  3.937369  3.322825  14.576775   

0         2149882   2151691    1226429  
3        8.736669  8.7040

In [32]:
# check start & end time 
passed = True
starttime=df2.iloc[0]['TIMESTAMP']
endtime=df2.iloc[-1]['TIMESTAMP']
timediff = (endtime - starttime)
nrows=len(df2.index)
numValidTimes = nrows
secs = np.array([x.timestamp() for x in df2['TIMESTAMP']])
secsdiff = secs[1:-1]-secs[0:-2]
sample_interval = np.nanmedian(secsdiff)
#sample_interval2 = timediff.seconds/(nrows-1)  
if timediff.seconds>4*60*60: # files should be no more than 4 hours
    print('Problem likely with start time. Filter out all data more than 4 hours before end')
    df2 = df2[df2['TIMESTAMP']>endtime-pd.to_timedelta(4, unit='h')]
    df2 = df2[df2['TIMESTAMP']<=endtime]
    numValidTimes = len(df2.index)
    passed = False

In [35]:
thisd = {}
thisd['filename']=os.path.basename(csvfile)
thisd['starttime']=starttime
thisd['endtime']=endtime
thisd['hours']=np.round(timediff.seconds/3600.0,2)
thisd['npts']=nrows
thisd['nRECS']=df2.iloc[-1]['RECORD']-df2.iloc[0]['RECORD']+1
thisd['Fs']=1/sample_interval
if thisd['nRECS']!=nrows:
    passed=False
if numValidTimes<nrows:
    passed=False
thisd['numValidTimes']=numValidTimes
#thisd['numTimeSkips']=len(gpsdf.index)/3
thisd['passed']=passed
print(thisd)

{'filename': '100hz_Sensors_100Hz1.pkl', 'starttime': Timestamp('2022-11-04 12:00:00.010000'), 'endtime': Timestamp('2022-11-04 16:00:00'), 'hours': 4.0, 'npts': 1440000, 'nRECS': 1440000, 'Fs': 100.00009536752259, 'numValidTimes': 1440000, 'passed': True}
